In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv


In [3]:
train=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv")
test=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv")

# Data Exploration

In [38]:
train.info()

train.OS.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14731 entries, 0 to 14730
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HomePage                         14578 non-null  float64
 1   HomePage_Duration                14581 non-null  float64
 2   LandingPage                      14578 non-null  float64
 3   LandingPage_Duration             14596 non-null  float64
 4   ProductDescriptionPage           14608 non-null  float64
 5   ProductDescriptionPage_Duration  14564 non-null  float64
 6   GoogleMetric:Bounce Rates        14580 non-null  float64
 7   GoogleMetric:Exit Rates          14602 non-null  float64
 8   GoogleMetric:Page Values         14599 non-null  float64
 9   SeasonalPurchase                 14581 non-null  float64
 10  Month_SeasonalPurchase           14587 non-null  object 
 11  OS                               14597 non-null  float64
 12  SearchEngine      

2.0    7832
1.0    3072
3.0    2987
4.0     572
8.0      96
6.0      26
7.0       6
5.0       6
Name: OS, dtype: int64

In [39]:
features=train.columns[:-1]

In [40]:
column_float=[]
column_cat=[]

for i in train[features].columns:
    if train[i].dtype=='float64':
        column_float.append(i)
    else:
        column_cat.append(i)
        
print("integer column:\n ",column_float,'\n')
print("string column: \n",column_cat)


integer column:
  ['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration', 'ProductDescriptionPage', 'ProductDescriptionPage_Duration', 'GoogleMetric:Bounce Rates', 'GoogleMetric:Exit Rates', 'GoogleMetric:Page Values', 'SeasonalPurchase', 'OS', 'SearchEngine', 'Zone', 'Type of Traffic', 'WeekendPurchase'] 

string column: 
 ['Month_SeasonalPurchase', 'CustomerType', 'Gender', 'Cookies Setting', 'Education', 'Marital Status']


In [41]:
x_train=train[features]
y_train=train["Made_Purchase"]

In [42]:
y_train.value_counts()

False    9065
True     5666
Name: Made_Purchase, dtype: int64

# Preprocessing

In [43]:
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import normalize

encode= OrdinalEncoder()
Kimputer=KNNImputer(n_neighbors=7, weights="uniform")

col_n=["LandingPage_Duration",'HomePage_Duration','ProductDescriptionPage_Duration','GoogleMetric:Bounce Rates','GoogleMetric:Exit Rates','GoogleMetric:Page Values']

def process_function(dt):
    dt[column_cat]=pd.DataFrame(encode.fit_transform(dt[column_cat]))
    dt[dt.columns]=Kimputer.fit_transform(dt)
    dt[col_n]=normalize(dt[col_n], norm='l2', axis=1, copy=True, return_norm=False)

In [44]:
process_function(x_train)
x_train

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,OS,SearchEngine,Zone,Type of Traffic,CustomerType,Gender,Cookies Setting,Education,Marital Status,WeekendPurchase
0,0.0,0.000000,0.0,0.000000,1.0,0.000000,0.707107,0.707107,0.0,0.0,...,4.0,1.0,9.0,3.0,2.0,2.0,1.0,2.000000,1.0,0.0
1,0.0,0.000000,0.0,0.000000,2.0,0.998450,0.018721,0.052419,0.0,0.0,...,3.0,2.0,2.0,4.0,2.0,0.0,1.0,3.000000,0.0,0.0
2,0.0,0.000000,0.0,0.000000,10.0,1.000000,0.000032,0.000080,0.0,0.0,...,3.0,3.0,1.0,4.0,2.0,0.0,0.0,3.000000,0.0,1.0
3,0.0,0.000000,0.0,0.000000,1.0,0.000000,0.707107,0.707107,0.0,0.4,...,2.0,4.0,3.0,3.0,2.0,1.0,0.0,0.000000,2.0,0.0
4,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.707107,0.707107,0.0,0.0,...,1.0,2.0,1.0,5.0,2.0,1.0,1.0,0.000000,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14726,1.0,0.004069,0.0,0.000000,39.0,0.999992,0.000016,0.000018,0.0,0.0,...,3.0,2.0,6.0,3.0,2.0,0.0,1.0,1.857143,0.0,0.0
14727,8.0,0.411438,2.0,0.200403,11.0,0.889133,0.000000,0.000039,0.0,0.0,...,2.0,2.0,2.0,4.0,2.0,2.0,0.0,0.000000,0.0,0.0
14728,2.0,0.057315,2.0,0.494910,10.0,0.867052,0.000000,0.000018,0.0,0.0,...,2.0,2.0,4.0,2.0,2.0,2.0,2.0,3.000000,2.0,0.0
14729,0.0,0.000000,0.0,0.000000,6.0,1.000000,0.000000,0.000032,0.0,0.0,...,2.0,4.0,4.0,1.0,2.0,2.0,2.0,2.000000,0.0,0.0


In [45]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
target_label = label_encoder.fit_transform(y_train)

In [46]:
target_label[:5]

array([0, 0, 0, 0, 0])

In [47]:
x_test=test[features]
process_function(x_test)
x_test

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,OS,SearchEngine,Zone,Type of Traffic,CustomerType,Gender,Cookies Setting,Education,Marital Status,WeekendPurchase
0,0.0,0.000000,0.0,0.000000,1.0,0.000000,7.071068e-01,0.707107,0.0,0.0,...,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,0.0
1,0.0,0.000000,0.0,0.000000,2.0,0.999999,0.000000e+00,0.001562,0.0,0.0,...,2.0,2.0,1.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0
2,0.0,0.000000,0.0,0.000000,19.0,1.000000,1.023850e-04,0.000159,0.0,0.0,...,2.0,2.0,1.0,3.0,2.0,0.0,2.0,0.0,1.0,0.0
3,0.0,0.000000,0.0,0.000000,2.0,0.999996,0.000000e+00,0.002703,0.0,0.8,...,2.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,0.0
4,0.0,0.000000,0.0,0.000000,16.0,1.000000,4.598406e-05,0.000063,0.0,0.4,...,1.0,1.0,4.0,3.0,2.0,0.0,1.0,3.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6594,0.0,0.000000,0.0,0.000000,7.0,1.000000,0.000000e+00,0.000137,0.0,0.0,...,4.0,1.0,1.0,5.0,2.0,2.0,2.0,1.0,1.0,1.0
6595,0.0,0.000000,3.0,0.025301,179.0,0.999680,1.569841e-08,0.000015,0.0,0.0,...,2.0,4.0,9.0,11.0,2.0,1.0,2.0,2.0,1.0,0.0
6596,5.0,0.073414,1.0,0.019988,33.0,0.997101,2.056426e-06,0.000007,0.0,0.6,...,8.0,5.0,1.0,2.0,2.0,0.0,2.0,1.0,0.0,0.0
6597,0.0,0.000000,0.0,0.000000,3.0,0.999863,7.406392e-03,0.014813,0.0,0.0,...,2.0,2.0,2.0,3.0,2.0,2.0,1.0,0.0,1.0,1.0


# Feature Selection

In [48]:
from sklearn.feature_selection import SelectKBest,chi2

#apply SelectKBest class to extract top 15 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(x_train,y_train)
scores = pd.DataFrame(fit.scores_)
columns = pd.DataFrame(x_train.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([columns,scores],axis=1)

featureScores.columns = ['Specs','Score']  #naming the dataframe columns
featureScores

print(featureScores.nlargest(15,'Score'))  #print 10 best features
new_x_train=bestfeatures.transform(x_train)

                              Specs        Score
4            ProductDescriptionPage  4371.957537
0                          HomePage   188.212123
2                       LandingPage    86.618145
8          GoogleMetric:Page Values    71.714204
7           GoogleMetric:Exit Rates    13.234541
10           Month_SeasonalPurchase    11.974682
6         GoogleMetric:Bounce Rates    11.823099
15                     CustomerType     5.736335
13                             Zone     3.168966
9                  SeasonalPurchase     3.011131
12                     SearchEngine     1.535362
20                  WeekendPurchase     1.320167
1                 HomePage_Duration     0.962338
14                  Type of Traffic     0.946779
5   ProductDescriptionPage_Duration     0.866472


# Splitting and Validation Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(new_x_train,target_label, test_size=0.2,random_state=42, stratify=y_train)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
from sklearn import Pipeline
from sklearn import Standard Scalar

pipe=[enc='StandardScalar',
     ]


In [ ]:
from sklearn.metrics import roc_auc_score, ConfusionMatrixDisplay, confusion_matrix

def get_metrics(classifier, Xvalid, yvalid):
    print("Validation Score : ", classifier.score(Xvalid, yvalid))
   
    y_valid_predict = classifier.predict(Xvalid)
    
    #Confusion Matrix
    cm = confusion_matrix(yvalid,y_valid_predict)
    dist = ConfusionMatrixDisplay(cm)
    dist.plot()
    plt.show()

# Dummy

In [ ]:
# from sklearn.dummy import DummyClassifier
# clf = DummyClassifier(strategy="most_frequent")
# clf.fit(x_train, y_train)
# y_pred = clf.predict(x_train)
# from sklearn.metrics import f1_score
# f1_score(y_train,y_pred)

# Logistic Regression
### highest score 0.67 without removing outliers

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(random_state=64, solver='newton-cg')
# clf.fit(X_train, y_train)
# print("Training Score : ", clf.score(X_train,y_train))

# # get matrix
# get_metrics(clf, X_valid,y_valid)

# Random Forest

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# model=RandomForestClassifier()
# model.fit(X_train,y_train)
# print("Training Score : ", model.score(X_train,y_train))

# # get matrix
# get_metrics(model, X_valid,y_valid)

# Disicion Tree

In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# x_test=test
# clf = DecisionTreeClassifier()

# # train the model on the training data
# clf.fit(X_train, y_train)

# # make predictions on the test data
# print("Training Score : ", clf.score(X_train,y_train))

# # get matrix
# get_metrics(clf, X_valid,y_valid)

# XGBClassifier

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

print("Training Score : ", xgb.score(X_train,y_train))

# get matrix
get_metrics(xgb, X_valid,y_valid)

# Hypertunning

## XGB Classifier

In [ ]:
params ={
    'max_depth': [3, 4, 5, 6, 8, 10, 12, 15],
    'learning_rate': [0.05,0.10,0.15,0.20,0.25,0.30],
    'n_estimators': [100, 200,300,400,500,600,900,1000,2000,4000,5000],
    'colsample_bytree': [0.3,0.4,0.5,0.7],
    'gamma':[0.0,0.1,0.2,0.3,0.4],
    'scale_pos_weight':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
    'max_delta_step':[0,1,2,3,4,5,6,7,8,9,10]
}

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

xgb = XGBClassifier()

random_search = RandomizedSearchCV(xgb, params, n_iter=10, scoring='roc_auc', n_jobs=-1, cv=7, verbose = 3)
random_search.fit(X_train, y_train)

print(random_search.best_params_)

In [ ]:
model_final = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6,
              enable_categorical=False, gamma=0.4, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.3, max_delta_step=10, max_depth=3,
              min_child_weight=47, monotone_constraints='()',
              n_estimators=1000, n_jobs=12, num_parallel_tree=1,
              predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1.2, subsample=0.7, tree_method='exact',
              validate_parameters=1, verbosity=None)

eval_set = [(X_valid, y_valid)]

model_final.fit(X_train,y_train,early_stopping_rounds=10,eval_set=eval_set,verbose=True)

print("Training Score: ", model_final.score(X_valid,y_valid))
get_metrics(model_final, X_valid, y_valid)

# Predicting for x_test

In [ ]:
final_test=bestfeatures.transform(x_test)
prediction = model_final.predict(final_test)


In [ ]:
predict=[]
for i in prediction:
    if i==1:
        predict.append(True)
    else:
        predict.append(False)

In [ ]:
sample=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv")

In [ ]:
submission=pd.DataFrame(columns=['id', 'Made_Purchase'])
submission['id']=[i for i in range(len(predict))]
submission["Made_Purchase"]=predict
submission.to_csv('submission.csv',index=False)